In [2]:
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import cv2 as cv2
from ultralytics import YOLO
import numpy as np

# Load the YOLO model
model = YOLO('yolov8l.pt')

def detect_objects(image):
    results = model(image)
    return results

In [4]:

# Load pre-trained face and gender classification models
face_prototxt_path = 'models/deploy.prototxt'
face_model_path = 'models/res10_300x300_ssd_iter_140000_fp16.caffemodel'

# Paths to gender classification model files
gender_prototxt_path = 'models/deploy_gender2.prototxt'
gender_model_path = 'models/gender_net (1).caffemodel'
face_net = cv2.dnn.readNetFromCaffe(face_prototxt_path, face_model_path)
gender_net = cv2.dnn.readNetFromCaffe(gender_prototxt_path, gender_model_path)
gender_list = ['Male', 'Female']

def detect_gender(face_img):
    blob = cv2.dnn.blobFromImage(face_img, 1.0, (227, 227), (104.0, 177.0, 123.0))
    gender_net.setInput(blob)
    gender_preds = gender_net.forward()
    gender = gender_list[gender_preds[0].argmax()]
    return gender


In [8]:


def change_car_color(image, bbox, label):
    x1, y1, x2, y2 = map(int, bbox)
    car = image[y1:y2, x1:x2]
    hsv_car = cv2.cvtColor(car, cv2.COLOR_BGR2HSV)
    
    red_lower = np.array([0, 120, 70])
    red_upper = np.array([10, 255, 255])
    mask_red = cv2.inRange(hsv_car, red_lower, red_upper)
    
    blue_lower = np.array([110, 50, 50])
    blue_upper = np.array([130, 255, 255])
    mask_blue = cv2.inRange(hsv_car, blue_lower, blue_upper)
    
    if np.any(mask_red):
        car[mask_red > 0] = [255, 0, 0]  # Change red to blue
    elif np.any(mask_blue):
        car[mask_blue > 0] = [0, 0, 255]  # Change blue to red
    
    image[y1:y2, x1:x2] = car
    return image





class TrafficApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Traffic Signal Analysis")

        self.canvas = tk.Canvas(root, width=800, height=600)
        self.canvas.pack()

        self.btn_load = tk.Button(root, text="Load Image", command=self.load_image)
        self.btn_load.pack()

        self.btn_process = tk.Button(root, text="Process Image", command=self.process_image)
        self.btn_process.pack()

    def load_image(self):
        self.filepath = filedialog.askopenfilename()
        if self.filepath:
            self.image = Image.open(self.filepath)
            self.photo = ImageTk.PhotoImage(self.image)
            self.canvas.create_image(0, 0, anchor=tk.NW, image=self.photo)

    def process_image(self):
        if hasattr(self, 'filepath'):
            image = cv2.imread(self.filepath)
            results = detect_objects(image)

            for result in results:
                for detection in result.boxes:
                    bbox = detection.xyxy[0].tolist()  # Get bounding box
                    label = detection.cls[0].item()  # Get label index
                    confidence = detection.conf[0].item()  # Get confidence score
                    
                    if label == 2:  # Assuming 'car' label is at index 2
                        image = change_car_color(image, bbox, label)
                    elif label == 0:  # Assuming 'person' label is at index 0
                        face_img = image[int(bbox[1]):int(bbox[3]), int(bbox[0]):int(bbox[2])]
                        gender = detect_gender(face_img)
                        print(f"Detected {gender}")

            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image_pil = Image.fromarray(image_rgb)
            self.photo = ImageTk.PhotoImage(image_pil)
            self.open_results_tab(self.photo)

    def open_results_tab(self, photo):
        new_window = tk.Toplevel(self.root)
        new_window.title("Processed Image")
        canvas = tk.Canvas(new_window, width=photo.width(), height=photo.height())
        canvas.pack()
        canvas.create_image(0, 0, anchor=tk.NW, image=photo)
        new_window.mainloop()

if __name__ == "__main__":
    root = tk.Tk()
    app = TrafficApp(root)
    root.mainloop()



0: 416x640 10 persons, 21 cars, 1 tie, 715.2ms
Speed: 2.9ms preprocess, 715.2ms inference, 2.0ms postprocess per image at shape (1, 3, 416, 640)
Detected Male
Detected Male
Detected Male
Detected Male
Detected Female
Detected Male
Detected Male
Detected Female
Detected Female
Detected Male
